In [1]:
import ipywidgets as widgets
import ipyvolume as ipv

import numpy as np

In [2]:
def get_relative_isotherms(v_range, T_range):
    
    """
    
        This function calculates the p(v, T) plane
        from given a and b parameters of 
        the Van der Waals equation of state for real gases.

    Args:
        a: Term related with the attraction between particles in
        J m^3/ mol^2.\n
        b: Term related with the volume that is occupied by one 
        mole of the molecules in 10^-3 m^3/mol.\n
        T_range: Tuple containing maximum and minimum values of
        temperature to be plotted. Temperatures must be expressed
        in terms of the critical temperature.\n
        v_range: Tuple containing maximum and minimum values of
        volumen to be plotted. Volumens must be expressed
        in exponents of the critical volumen (v in 10^v_range).\n
        
    Returns:
        isotherms: a dictionary containing the values of v and the
        pressures of the isotherms in the following form:\n
            isotherms['v'] = np.array containing the values of v
            in a logarithmic scale.\n
            isotherms['value of the isotherm'] = np.array containing
            the values of p.\
    """

    isotherms = []

    for T in T_range:
        p_R = []
        for v in v_range:
            val = (8.0/3.0*T/(v - 1.0/3.0) - 3.0/v**2)
            p_R = np.append(p_R, val)

        isotherms.append(p_R)

    return isotherms

In [3]:
T_values = np.linspace(0.8, 1.0, 10)
v_values = np.linspace(0.45, 5.0, 500)

p_values = get_relative_isotherms(v_values, T_values)

In [4]:
def get_roots(p, T):
    
    roots = np.roots([1.0, - 1.0/3.0*(1.0 + 8.0*T/p), 3.0/p, -1.0/p])
    roots_in_range = []
    
    for root in roots:
        if np.isreal(root):
            root = np.real(root)
            if root > 0:
                roots_in_range.append(root)
    #print(roots_in_range) 
    roots_in_range.sort()
    
    return roots_in_range

In [5]:
def find_real_fixed_T(p_range, T):
    
    eps = 1e-3 #1.0/p_range.size
    
    for p in p_range:
        
        roots = get_roots(p, T)
        if len(roots) == 3:
            v_range = (roots[0], roots[2])
            area = defined_integral(p, v_range, T)
            if abs(area) < eps:
                return p, v_range

    return None

In [6]:
fig3d = ipv.pylab.figure(key=None,
                       width=600,
                       height=500,
                       lighting=True,
                       controls=True,
                       controls_vr=False,
                       controls_light=False,
                       debug=False,
                       )
ipv.pylab.xlim(min(v_values), max(v_values))
ipv.pylab.ylim(0.0, 2.0)
ipv.pylab.zlim(min(T_values), max(T_values))

ipv.pylab.xlabel('v')
ipv.pylab.ylabel('p')
ipv.pylab.zlabel('T')

ipv.pylab.view(azimuth=180, elevation=None, distance=None)

for i in range(len(T_values)):
    
    x,y,z = np.asarray(v_values), np.asarray(p_values[i]), np.asarray([T_values[i] for elem in v_values])
    ipv.pylab.plot(x, y, z)

In [7]:
def update_camera_angle(change):
    ipv.pylab.view(azimuth=degrees_slider.value)

In [8]:
degrees_slider = widgets.IntSlider(
    value=180,
    min=0,
    max=360,
    step=1,
    description='',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

degrees_slider.observe(update_camera_angle, 'value')

In [9]:
play = widgets.Play(
#     interval=10,
    value=180,
    min=0,
    max=360,
    step=1,
    description="Press play",
    disabled=False
)
widgets.jslink((play, 'value'), (degrees_slider, 'value'));

In [10]:
widgets.VBox([fig3d, widgets.HBox([play, degrees_slider])], layout=widgets.Layout(align_items='center'))